In [193]:
import pandas as pd
import numpy as np
import uproot
from schema import Schema, And, Use, Optional, SchemaError

In [213]:
def angle(v1: np.array, v2: np.array) -> float:
    """ Calculates the angle between two vectors in radians

    Parameters
    ----------
    v1: np.array
        vector 1
    v2: np.array
        vector 2

    Returns
    -------
    angle: float
        The calculates angle in radians
    """
    angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))
    return angle

def p2azimuthAndzenith(p: np.array):
    """ converts a momentum vector to azimuth and zenith angles

    Parameters
    ----------
    p: np.array
        The 3d momentum

    Returns
    -------
    float, float:
        The azimuth and zenith angles in radians.
    """
    azimuth = angle(p[:2], np.array([0, 1]))
    zenith = angle(p[1:], np.array([0., 1]))
    return azimuth, zenith

In [214]:
def genie_parser(events)->pd.DataFrame:
    """ function to fetch the relevant information from genie events (in rootracker format)

    Parameters
    ----------
    events : dict
        The genie events

    Returns
    -------
    pd.DataFrame
        Data frame containing the relevant information
    """
    dic = {}
    dic['event_description'] = events['EvtCode/fString'].array(library='np')  # String describing the event
    dic['event_id'] = events['EvtNum'].array(library="np")  # Number of the event
    dic['event_children'] = events['StdHepN'].array(library="np")  # NUmber of the generated sub-particles
    dic['event_prob'] = events['EvtProb'].array(library="np")  # Probability of the event
    dic['event_xsec'] = events['EvtXSec'].array(library="np")  # Total xsec of the event
    dic['event_pdg_id'] = events['StdHepPdg'].array(library="np")  # PDG ids of all produced particles
    dic['event_momenta'] = events['StdHepP4'].array(library="np")  # Momenta of the particles
    tmp = events['EvtVtx'].array(library="np")  # Position of the particles
    dic['event_vertex'] = [np.array(vtx) for vtx in tmp]
    dic['event_coords'] = events['StdHepX4'].array(library="np")  # Position of the particles
    dic['event_weight'] = events['EvtWght'].array(library='np')  # Weight of the events
    tmp = events['StdHepStatus'].array(library="np")  # Status of the particle
    # Converting the codes
    particle_dic = {
        0: 'initial',
        1: 'final',
        2: 'intermediate',
        3: 'decayed',
        11: 'nucleon target',
        12: 'DIS pre-frag hadronic state',
        13: 'resonance',
        14: 'hadron in nucleus',
        15: 'final nuclear',
        16: 'nucleon cluster target',
    }
    new_arr = np.array([[
        particle_dic[particle] for particle in event
    ] for event in tmp], dtype=object)
    dic['event_status'] = new_arr
    return pd.DataFrame.from_dict(dic)

def final_parser(parsed_events: pd.DataFrame)->pd.DataFrame:
    """ fetches the final states

    Parameters
    ----------
    parsed_events : pd.DataFrame
        The parsed events

    Returns
    -------
    pd.DataFrame
        The inital + final state info
    """
    inital_energies_inj = np.array([event[0][3] for event in parsed_events['event_momenta']])
    inital_momenta_inj = [np.array(event[0][:3]) for event in parsed_events['event_momenta']]
    inital_energies_target = np.array([event[1][3] for event in parsed_events['event_momenta']])
    inital_id_inj = np.array([event[0] for event in parsed_events['event_pdg_id']])
    inital_id_target = np.array([event[1] for event in parsed_events['event_pdg_id']])
    final_ids = np.array([np.where(event == np.array('final'), True, False) for event in parsed_events['event_status']], dtype=object)
    children_ids = np.array([
        event[final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_pdg_id'])
    ], dtype=object)
    children_energy = np.array([
        event[:, 3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    children_momenta = np.array([
        event[:, :3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    final_ids = np.array([np.where(event == np.array('final nuclear'), True, False) for event in parsed_events['event_status']], dtype=object)
    children_nuc_ids = np.array([
        event[final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_pdg_id'])
    ], dtype=object)
    children_nuc_energy = np.array([
        event[:, 3][final_ids[id_event]] for id_event, event in enumerate(parsed_events['event_momenta'])
    ], dtype=object)
    dic = {}
    dic['event_descr'] = parsed_events['event_description']
    dic['event_xsec'] = parsed_events['event_xsec']
    dic['event_vertex'] = parsed_events.event_vertex
    dic['init_inj_e'] = inital_energies_inj
    dic['init_inj_p'] = inital_momenta_inj
    dic['init_target_e'] = inital_energies_target
    dic['init_inj_id'] = inital_id_inj
    dic['init_target_id'] = inital_id_target
    dic['final_ids'] = children_ids
    dic['final_e'] = children_energy
    dic['final_p'] = children_momenta
    dic['final_nuc_ids'] = children_nuc_ids
    dic['final_nuc_e'] = children_nuc_energy
    dic['p4'] = np.array([event for event in parsed_events['event_momenta']], dtype='object')
    return pd.DataFrame.from_dict(dic)

In [215]:
with uproot.open('gtrac_1_nue.root') as file:
    events = file['gRooTracker']
    parsed_events = genie_parser(events)
    final_parsed = final_parser(parsed_events)

In [216]:
final_parsed.iloc[0]

event_descr          nu:14;tgt:1000060120;N:2112;proc:Weak[CC],QES;
event_xsec                                                 2.417622
event_vertex                                   [0.0, 0.0, 0.0, 0.0]
init_inj_e                                                 0.434245
init_inj_p                           [0.0, 0.0, 0.4342451003287754]
init_target_e                                             11.174863
init_inj_id                                                      14
init_target_id                                           1000060120
final_ids                                                [13, 2212]
final_e                    [0.2812121215868135, 1.0654573858712357]
final_p           [[0.1933804519351921, -0.14307357003140753, 0....
final_nuc_ids                                          [2000000002]
final_nuc_e                                    [10.262438692870726]
p4                [[0.0, 0.0, 0.4342451003287754, 0.434245100328...
Name: 0, dtype: object

In [247]:
all_fields = [
            "injection_energy",
            "injection_type",
            "injection_interaction_type",
            "injection_zenith",
            "injection_azimuth",
            "injection_bjorkenx",
            "injection_bjorkeny",
            "injection_position_x",
            "injection_position_y",
            "injection_position_z",
            "injection_column_depth",
            "primary_particle_1_type",
            "primary_particle_1_position_x",
            "primary_particle_1_position_y",
            "primary_particle_1_position_z",
            "primary_particle_1_direction_theta",
            "primary_particle_1_direction_phi",
            "primary_particle_1_energy",
            "primary_particle_2_type",
            "primary_particle_2_position_x",
            "primary_particle_2_position_y",
            "primary_particle_2_position_z",
            "primary_particle_2_direction_theta",
            "primary_particle_2_direction_phi",
            "primary_particle_2_energy",
            "total_energy",
        ]

def schema_check(std_schema: Schema, to_check)->bool:
    """ quick and dirty function to validate formatting
    """
    try:
        std_schema.validate(to_check)
        return True
    except SchemaError:
        return False
# inj_particle_scheme = Schema({
#         "primary": And(Use(bool)),  # If this thing is a primary particle
#         "energy": And(Use(float)),  # The energy in GeV,  # NOTE: Should this be kinetic energy?
#         "PDG": And(Use(int)),  # The PDG id
#         "interaction": And(Use(str)),  # Should we use ints to define interactions or strings?
#         "theta": And(Use(float)),  # It's zenith angle
#         "phi": And(Use(float)),  # It's azimuth angle
#         "bjorken_x": And(Use(float)),  # Bjorken x variable
#         "bjorken_y": And(Use(float)),  # Bjorken y variable
#         "pos_x": And(Use(float)),  # position x (in detector coordinates)
#         "pos_y": And(Use(float)),  # position y (in detector coordinates)
#         "pos_z": And(Use(float)),  # position z (in detector coordinates)
#         "column_depth": And(Use(float)),  # Column depth where the interaction happened
#         Optional('custom_info'): And(Use(str))  # Additional information the user can add as a string. # TODO: This should be handed to the propagators
#     })

def genie2prometheus(parsed_events: pd.DataFrame):
    """ reformats parsed GENIE events into a usable format for PROMETHEUS
    NOTES: Create a standardized scheme function. This could then be used as an interface to PROMETHEUS
    for any injector. E.g. a user would only need to create a function to translate their injector output to the scheme format.

    Parameters
    ----------
    parsed_events: pd.DataFrame
        Dataframe object containing all the relevant (and additional) information needed by PROMETHEUS

    Returns
    -------
    particles: pd.DataFrame
        Fromatted data set with values which can be used directly.
    injection: pd.Dataframe
        The injected particle in the same format
    """
    # TODO: Use a more elegant definition to couple to the particle class
    primaries = {}
    event_set  = {}
    for index, event in parsed_events.iterrows():
        if 'CC' in event.event_descr:
            event_type = 'CC'
        elif 'NC' in event.event_descr:
            event_type = 'NC'
        else:
            event_type = 'other'
        azimuth, zenith =  p2azimuthAndzenith(event.init_inj_p)
        primary_particle = {
            "primary": True,  # If this thing is a primary particle
            "e": event.init_inj_e,  # The energy in GeV,  # NOTE: Should this be kinetic energy?
            "pdg_code": event.init_inj_id,  # The PDG id
            "interaction": event_type,  # Should we use ints to define interactions or strings?
            "theta": zenith,  # It's zenith angle
            "phi": azimuth,  # It's azimuth angle
            "bjorken_x": -1,  # Bjorken x variable
            "bjorken_y": -1,  # Bjorken y variable
            "pos_x": event.event_vertex[0],  # position x (in detector coordinates)
            "pos_y": event.event_vertex[1],  # position y (in detector coordinates)
            "pos_z": event.event_vertex[2],  # position z (in detector coordinates)
            "position": event.event_vertex[:3],  # 3d position
            "column_depth": -1,  # Column depth where the interaction happened
            'custom_info': event.event_descr  # Additional information the user can add as a string. # TODO: This should be handed to the propagators
        }
        # TODO: Optimize
        angles = np.array([p2azimuthAndzenith(p) for p in event.final_p])
        particles = {
            "primary": np.array(np.zeros(len(event.final_ids)), dtype=bool),  # If this thing is a primary particle
            "e": event.final_e,  # The energy in GeV,  # NOTE: Should this be kinetic energy?
            "pdg_code": event.final_ids,  # The PDG id
            "interaction": np.array([event_type for _ in range(len(event.final_ids))]),  # Should we use ints to define interactions or strings?
            "theta": angles[:, 1],  # It's zenith angle
            "phi": angles[:, 0],  # It's azimuth angle
            "bjorken_x": np.ones(len(event.final_ids)) * -1,  # Bjorken x variable
            "bjorken_y": np.ones(len(event.final_ids)) * -1,  # Bjorken y variable
            "pos_x": np.ones(len(event.final_ids)) * event.event_vertex[0],  # position x (in detector coordinates)
            "pos_y": np.ones(len(event.final_ids)) * event.event_vertex[1],  # position y (in detector coordinates)
            "pos_z": np.ones(len(event.final_ids)) * event.event_vertex[2],  # position z (in detector coordinates)
            "position": np.array([event.event_vertex[:3] for _ in range(len(event.final_ids))]),  # 3d position
            "column_depth": np.ones(len(event.final_ids)) * -1,  # Column depth where the interaction happened
            'custom_info': np.array(['child' for _ in range(len(event.final_ids))])  # Additional information the user can add as a string. # TODO: This should be handed to the propagators
        }
        event_set[index] = particles
        primaries[index] = primary_particle
    return pd.DataFrame.from_dict(event_set, orient='index'), pd.DataFrame.from_dict(primaries, orient='index')


In [248]:
prometheus_set, primary_set = genie2prometheus(final_parsed)

/tmp/ipykernel_3490/4033992054.py:16: RuntimeWarning: invalid value encountered in double_scalars
  angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))


In [249]:
prometheus_set

,primary,e,pdg_code,interaction,theta,phi,bjorken_x,bjorken_y,pos_x,pos_y,pos_z,position,column_depth,custom_info
0,"[False, False]","[0.2812121215868135, 1.0654573858712357]","[13, 2212]","[CC, CC]","[0.9595977685828917, 0.7848725433119043]","[2.20777320930952, 0.3722303358621821]","[-1.0, -1.0]","[-1.0, -1.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[-1.0, -1.0]","[child, child]"
1,"[False, False]","[2.2262342103757886, 1.0112779131884424]","[13, 2212]","[CC, CC]","[0.020232228695744592, 0.6284253781509385]","[1.4277499846093367, 1.7931375031855128]","[-1.0, -1.0]","[-1.0, -1.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[-1.0, -1.0]","[child, child]"
2,"[False, False, False, False]","[0.4810411233983117, 1.4129668406657494, 1.042...","[13, 2112, 2112, 2212]","[CC, CC, CC, CC]","[0.24948517893938313, 0.07330731731301666, 2.9...","[1.8448727846663373, 1.4229307904310187, 1.251...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
3,"[False, False, False]","[0.2781896818998354, 1.0820755909413229, 1.451...","[13, 2212, 111]","[CC, CC, CC]","[1.6544641693506619, 0.07907969977958809, 0.19...","[2.3501156979836724, 1.3194829858440122, 0.229...","[-1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0]","[child, child, child]"
4,"[False, False, False, False]","[2.7728271136055214, 1.8034664795646638, 0.752...","[14, 2112, 211, 111]","[NC, NC, NC, NC]","[0.02682544733747358, 0.28739287387775886, 0.3...","[1.4806606884136464, 2.0460199710899785, 0.760...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"[False, False, False, False, False]","[0.9938833457498917, 1.5063703777383424, 0.810...","[13, 2212, -211, 211, 111]","[CC, CC, CC, CC, CC]","[0.2620187896727822, 0.3619675914726309, 0.119...","[1.2652738193982842, 2.383750429030567, 1.2360...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0, -1.0]","[child, child, child, child, child]"
996,"[False, False, False]","[0.9628922545889975, 0.9552543287071839, 0.956...","[13, 2112, 2212]","[CC, CC, CC]","[0.09690579149898174, 0.43121035229273613, 0.3...","[2.439109073450234, 2.3700282698795436, 0.8385...","[-1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0]","[child, child, child]"
997,"[False, False, False, False]","[3.321388778339659, 1.982438765532098, 1.01434...","[13, 2212, 2112, 211]","[CC, CC, CC, CC]","[0.3122597257184119, 0.35271664415190906, 0.98...","[0.2171831324464813, 3.135607894892876, 2.1853...","[-1.0, -1.0, -1.0, -1.0]","[-1.0, -1.0, -1.0, -1.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...","[-1.0, -1.0, -1.0, -1.0]","[child, child, child, child]"
998,"[False, False]","[1.0180434386141433, 2.3274945143607604]","[13, 2212]","[CC, CC]","[0.9805755132559603, 0.33460407814609444]","[2.5961896718044293, 0.590286735703562]","[-1.0, -1.0]","[-1.0, -1.0]","[0.0, 0.0]","[0.0, 0.0]","[0.0, 0.0]","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]","[-1.0, -1.0]","[child, child]"
